In [ ]:
import cv2
import numpy as np
import os

def extract_objects(image_path, output_dir, min_area=100, max_area=None):
    """
    Extract multiple objects from an image with white background.
    
    Parameters:
    - image_path: Path to the input image
    - output_dir: Directory to save extracted objects
    - min_area: Minimum contour area to consider (filters out noise)
    - max_area: Maximum contour area to consider (None for no limit)
    
    Returns:
    - List of bounding boxes for extracted objects
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Could not read the image")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Threshold the image to create a binary mask
    # Assuming white background, adjust threshold as needed
    _, binary = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)
    
    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Store bounding boxes
    bounding_boxes = []
    
    # Process each contour
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        
        # Skip if area is too small or too large
        if area < min_area:
            continue
        if max_area and area > max_area:
            continue
            
        # Get bounding box
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, w, h))
        
        # Create mask for this contour
        mask = np.zeros_like(binary)
        cv2.drawContours(mask, [contour], -1, 255, -1)
        
        # Extract object using the mask
        object_image = image.copy()
        object_image[mask == 0] = 255  # Set background to white
        
        # Crop to bounding box
        object_image = object_image[y:y+h, x:x+w]
        
        # Save extracted object
        output_path = os.path.join(output_dir, f'object_{i}.png')
        cv2.imwrite(output_path, object_image)
    
    return bounding_boxes

def visualize_results(image_path, bounding_boxes):
    """
    Visualize detected objects with bounding boxes.
    
    Parameters:
    - image_path: Path to the original image
    - bounding_boxes: List of (x, y, w, h) tuples
    """
    image = cv2.imread(image_path)
    
    # Draw bounding boxes
    for x, y, w, h in bounding_boxes:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Display result
    cv2.imshow('Detected Objects', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

def extract_objects(image_path):
    # Read image
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    
    # Convert to grayscale if not already
    if len(img.shape) > 2:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    
    # Apply adaptive threshold
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 cv2.THRESH_BINARY_INV, 11, 2)
    
    # Create kernel for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    
    # Close small gaps
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    # Find connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed, 
                                                                          connectivity=8)
    
    # Filter components based on size if needed
    min_size = 50  # adjust this value based on your needs
    objects = []
    
    for i in range(1, num_labels):  # Skip background (label 0)
        if stats[i, cv2.CC_STAT_AREA] >= min_size:
            # Create mask for this component
            mask = np.zeros_like(gray)
            mask[labels == i] = 255
            
            # Dilate mask slightly to ensure we capture semi-transparent edges
            mask = cv2.dilate(mask, kernel, iterations=1)
            
            # Extract object using mask
            object_img = cv2.bitwise_and(img, img, mask=mask)
            objects.append(object_img)
    
    return objects

# Usage
image_path = r"C:\Users\acer\Desktop\Work_IGB\Georgia Zooplankton\Zooplankton scanner raw and sorted images\Zooplankton scanner raw and sorted images\M3A_2011-08-27__45um_above200um_x1_2400dpi_1-of-3.jpg"

objects = extract_objects(image_path)

MemoryError: Unable to allocate 75.0 MiB for an array with shape (8869, 8869) and data type uint8

In [5]:
image_path = r"C:\Users\acer\Desktop\Work_IGB\Georgia Zooplankton\Zooplankton scanner raw and sorted images\Zooplankton scanner raw and sorted images\M3A_2011-08-27__45um_above200um_x1_2400dpi_1-of-3.jpg"
output_dir = "extracted_objects"

try:
    # Extract objects
    boxes = extract_objects(
        image_path=image_path,
        output_dir=output_dir,
        min_area=50  # Adjust based on your needs
    )
    
    print(f"Successfully extracted {len(boxes)} objects")
    
    # Visualize results
    visualize_results(image_path, boxes)
    
except Exception as e:
    print(f"Error: {str(e)}")

Successfully extracted 42 objects


In [2]:
import cv2
import numpy as np
import os

image_path = r"C:\Users\acer\Desktop\Work_IGB\Georgia Zooplankton\Zooplankton scanner raw and sorted images\Zooplankton scanner raw and sorted images\M3A_2011-08-27__45um_above200um_x1_2400dpi_1-of-3.jpg"
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('Show', binary)
cv2.waitKey(0)
cv2.destroyAllWindows()